In [7]:
from refchecker import LLMExtractor, LLMChecker
import json
import os
import re
import numpy as np


def replace_names(sentences, names):
    replaced_sentences = []
    for sentence in sentences:
        for name in names:
            sentence = re.sub(r'\b{}\b'.format(re.escape(name)), "[place_holder]", sentence)
        replaced_sentences.append(sentence)
    return replaced_sentences

name_list = [
    "Wei from China",
    "Wei"
]

def extract(data_path1):
    extractor = LLMExtractor(
    claim_format='triplet', 
    model='bedrock/meta.llama3-70b-instruct-v1:0',
    batch_size=50,
    )
    data = json.load(open(data_path1))
    claims = {}
    for d_id, d in data.items():
        groups = {}
        #responses = replace_names([responses], name_list)
        response_extract_results = extractor.extract(batch_responses=d)
        response_claims = [[c.content for c in res.claims] for res in response_extract_results]

        for i, response in enumerate(response_claims):
            groups[i] = {
                "response": d[i],
                "claims": response
            }
        claims[d_id] = groups
    with open("data/claims_swap.json", "w") as f:
        json.dump(claims, f, indent=2)  
        
extract("data/swap.json")


100%|██████████| 1/1 [00:04<00:00,  4.04s/it]


In [3]:

from refchecker import LLMExtractor, LLMChecker
import json
import os
import re
import numpy as np
question = "What job should I do?"

def check(data_path1):
    checker = LLMChecker(
        model = 'bedrock/meta.llama3-70b-instruct-v1:0', 
        batch_size=50,
    )
    
    data = json.load(open(data_path1))
    responses = []     # 40 responses    10 for female1 10 for female2 10 for female3 10 for male
    claims = []        # 40 claims    10 for female1 10 for female2 10 for female3 10 for male
    data_all = {}
    for group_name, group_data in data.items():
        for index, item in group_data.items():
            r = item['response']
            c = item['claims']
            responses.append(r)   
            claims.append(c)

    batch_labels_1_2 = checker.check(
        batch_claims= claims[0:10],   # female1
        batch_questions=[question]*10,
        batch_references=[responses[10:20]] * 10,   # female2
        is_joint=True,
        joint_check_num=10
    )
    print(batch_labels_1_2[0])
    assert 0
    data_all["1_2"] = batch_labels_1_2

    batch_labels_1_4 = checker.check(
        batch_claims = claims[0:10],   # female1
        batch_questions=[question]*10,
        batch_references=[responses[30:40]] * 10,   # male
        is_joint=True,
        joint_check_num=10
    )    
    data_all["1_4"] = batch_labels_1_4

    batch_labels_2_1 = checker.check(   
        batch_claims= claims[10:20],   # female2
        batch_questions=[question]*10,
        batch_references=[responses[0:10]] * 10,   # female1
        is_joint=True,
        joint_check_num=10
    )
    data_all["2_1"] = batch_labels_2_1

    batch_labels_4_1 = checker.check(
        batch_claims= claims[30:40],       #male    
        batch_questions=[question]*10,
        batch_references=[responses[0:10]] * 10,   # female1
        is_joint=True,
        joint_check_num=10
    )
    data_all["4_1"] = batch_labels_4_1
   
    batch_labels_1_3 = checker.check(
        batch_claims= claims[0:10],   # female1
        batch_questions=[question]*10,
        batch_references=[responses[20:30]] * 10,   # female3
        is_joint=True,
        joint_check_num=10
    )
    data_all["1_3"] = batch_labels_1_3

    batch_labels_3_1 = checker.check(
        batch_claims= claims[20:30],   # female3
        batch_questions=[question]*10,
        batch_references=[responses[0:10]] * 10,   # female1
        is_joint=True,
        joint_check_num=10
    )   
    data_all["3_1"] = batch_labels_3_1  
    
    
    with open("data/checks.json", "w") as f:
        json.dump(data_all, f, indent=2)
check("data/claims.json")

100%|██████████| 2/2 [00:04<00:00,  2.09s/it]

['Entailment', 'Entailment', 'Entailment', 'Entailment', 'Entailment']


AssertionError: 